# 3.4章　記憶を持つエージェント

In [ ]:
!pip install langchain
!pip install langchain-openai

!pip install serpapi
!pip install google-search-results

# load_toolsを利用するのに必要
!pip install langchain_community

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')

# 3.4章

In [ ]:
from langchain_core.prompts import PromptTemplate

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools']
template="""\
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(input_variables=input_variables, template=template)
print(prompt.template)

In [ ]:
from langchain.memory import ChatMessageHistory

store = {}

def get_by_session_id(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import\
RunnableWithMessageHistory

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,\
verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_by_session_id,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = agent_with_chat_history.invoke({"input": "株式会社\
Elithの住所を教えてください。最新の公式情報として公開されているものを教え\
てください。"},
    config={"configurable": {"session_id": "test-session1"}})

In [ ]:
print(get_by_session_id("test-session1"))

In [ ]:
response = agent_with_chat_history.invoke({"input": "先ほど尋ね\
た会社は何の会社ですか？"},
    config={"configurable": {"session_id": "test-session1"}})

In [ ]:
print(get_by_session_id("test-session1"))

In [ ]:
print(get_by_session_id("test-session2"))

In [ ]:
response = agent_with_chat_history.invoke({"input": "先ほど尋ね\
た会社は何の会社ですか？"},
    config={"configurable": {"session_id": "test-session2"}})

In [ ]:
print(get_by_session_id("test-session2"))